In [1]:
from langchain_community.document_loaders import DirectoryLoader, JSONLoader

def metadata_func(record, metadata):
    metadata.update({
        "url": record.get("url", ""),
        "title": record.get("title", ""),
        "section": record.get("section", ""),
        "chunk_index": record.get("chunk_index", ""),
    })
    return metadata

loader = DirectoryLoader(
    "./data",
    glob="attack_on_Titan_Namu*.jsonl",
    loader_cls=JSONLoader,
    loader_kwargs={
        "jq_schema": ".",
        "json_lines": True,
        "content_key": "text",
        "metadata_func": metadata_func,
    },
)

document_list = loader.load()


In [2]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-small')

In [ ]:
# from langchain_chroma import Chroma

# # 데이터를 처음 저장할 때 
# database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='attack_on_Titan', persist_directory="./attack_on_Titan")


In [3]:
from langchain_chroma import Chroma

database = Chroma(
    collection_name="attack_on_Titan",
    persist_directory="./attack_on_Titan",
    embedding_function=embedding,
)

In [ ]:
from langchain_openai import ChatOpenAI

retriever = database.as_retriever(search_kwargs={"k": 3})
llm = ChatOpenAI(model="gpt-4o")

query = "attack titan의 역대 계승자에 대해 알려줘. 이 때 각각의 인물에 대해 자세하게 설명해봐"
docs = retriever.invoke(query)

context = "\n\n".join([d.page_content for d in docs])

prompt = f"""You are a helpful assistant. Use the context to answer.
질문: {query}

컨텍스트:
{context}
"""

response = llm.invoke(prompt)
response.content


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

llm = ChatOpenAI(model="gpt-4o")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use the context to answer."),
    ("human", "질문: {input}\n\n컨텍스트:\n{context}")
])

combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

ai_message = retrieval_chain.invoke({"input": query})
ai_message["answer"]


In [ ]:
# RAG: reduce hallucination by stricter retrieval + prompt
from langchain_openai import ChatOpenAI

# retriever with more candidates + MMR for diversity
retriever = database.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 24}
)

query = "아홉거인에 대해 자세하게 설명해줘"
docs = retriever.invoke(query)

# if retrieval is weak, fail fast
if len(docs) < 2:
    raise ValueError("컨텍스트가 부족합니다. 질문을 구체화하거나 k를 늘려주세요.")

context = "\n\n".join([d.page_content for d in docs])

prompt = f"""You are a helpful assistant.
Use ONLY the provided context.
If the answer is not in the context, say "정보 부족".


질문: {query}

컨텍스트:
{context}
"""

llm = ChatOpenAI(model="gpt-5-mini")
response = llm.invoke(prompt)
response.content


"다음 내용은 제공된 문맥 내에 있는 정보만을 정리한 것입니다. 그 밖의 상세한 설명은 문맥에 없습니다 — 정보 부족.\n\n- 아홉 거인(Nine Titans)에 대한 설명은 별도의 '아홉 거인' 문서를 참고하라고 되어 있습니다.\n- 계승자(또는 관련 인물)로 문맥에 나열된 항목들:\n  - 아홉 거인, 가비 브라운, 나일 도크, 다이나 프리츠, 도트 픽시스, 레온하트 씨, 로그(진격의 거인), 로드 레이스(진격의 거인), 맘몬(진격의 거인), 바리스(진격의 거인), 벽의 거인, 아홉 거인, 오거(진격의 거인)\n- 완전한 시조의 거인이 아홉 거인의 모든 힘을 사용할 수 있게 되면서 그 힘으로 모든 거인들의 경질화(갑옷화)를 해제하였다(문맥 내용). 그 결과 파라디 섬 세 방벽 내부의 초대형(벽 속) 거인들이 진격하여 방벽 주변 건물들이 무너지고 많은 사상자가 발생하기 시작했다는 기술이 있습니다.\n- 아홉 거인의 능력들이 더해지면 더욱 강력해지며, 예로 갑옷의 경질화와 전퇴의 능력, 소유자의 극한 숙련도가 합쳐진 후반부의 진격(아마 특정 거인 또는 상태)은 아홉 거인들 중 상위권 전투력으로 평가된다고 문맥에 서술되어 있습니다.\n- 추가 세부사항은 엘런 예거 문서의 8.1번 문단과 아홉 거인 문서를 참고하라고 명시되어 있습니다.\n\n위 외의 구체적 능력, 각 거인의 상세 설명 등은 문맥에 포함되어 있지 않습니다 — 정보 부족."